In [11]:
# The code was removed by Watson Studio for sharing.

# **Launch Sites Locations Analysis with Folium**


Estimated time needed: **40** minutes


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using `Folium`.


## Objectives


This lab contains the following tasks:

*   **TASK 1:** Mark all launch sites on a map
*   **TASK 2:** Mark the success/failed launches for each site on the map
*   **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, you should be able to find some geographical patterns about launch sites.


Let's first import required Python packages for this lab:


In [12]:
!pip3 install folium

In [13]:
import folium
import pandas as pd

In [14]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

## Task 1: Mark all launch sites on a map


First, let's try to add each site's location on a map using site's latitude and longitude coordinates


In [15]:
# Download and read the `spacex_launch_geo.csv`
spacex_df = pd.read_csv(project.get_file('data_wrangling.csv'))

Now, you can take a look at what are the coordinates for each site.


In [16]:
spacex_df

,FlightNumber,Date,BoosterVersion,PayloadMass,Orbit,PayloadName,Customer,Outcome,Flights,GridFins,...,Legs,LandingPad,Block,Serial,ReusedCount,LaunchSite,Longitude,Latitude,Block_Version,Class
0,1,2010-06-04,Falcon 9,7407.0,LEO,Dragon Qualification Unit,SpaceX,None None,1,False,...,False,NaN,1.0,B0003,0,CCSFS SLC 40,-80.577366,28.561857,v1.0,0
1,2,2012-05-22,Falcon 9,525.0,LEO,COTS Demo Flight 2,NASA(COTS),None None,1,False,...,False,NaN,1.0,B0005,0,CCSFS SLC 40,-80.577366,28.561857,v1.0,0
2,3,2013-03-01,Falcon 9,677.0,ISS,CRS-2,NASA (CRS),None None,1,False,...,False,NaN,1.0,B0007,0,CCSFS SLC 40,-80.577366,28.561857,v1.0,0
3,4,2013-09-29,Falcon 9,500.0,PO,CASSIOPE,MDA,False Ocean,1,False,...,False,NaN,1.0,B1003,0,VAFB SLC 4E,-120.610829,34.632093,v1.0,0
4,5,2013-12-03,Falcon 9,3170.0,GTO,SES-8,SES,None None,1,False,...,False,NaN,1.0,B1004,0,CCSFS SLC 40,-80.577366,28.561857,v1.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127,128,2022-01-13,Falcon 9,7407.0,LEO,NROL-87,National Reconnaissance Office,True RTLS,10,True,...,True,LZ-1,5.0,B1058,9,CCSFS SLC 40,-80.577366,28.561857,B5,1
128,129,2022-01-19,Falcon 9,13260.0,VLEO,Starlink 4-6 (v1.5),SpaceX,True ASDS,10,True,...,True,ASOG,5.0,B1060,9,KSC LC 39A,-80.603956,28.608058,B5,1
129,130,2022-01-31,Falcon 9,2240.0,SSO,CSG-2,ESA,True RTLS,3,True,...,True,LZ-1,5.0,B1052,2,CCSFS SLC 40,-80.577366,28.561857,B5,1
130,131,2022-02-02,Falcon 9,7407.0,LEO,NROL-87,National Reconnaissance Office,True RTLS,1,True,...,True,LZ-4,5.0,B1071,0,VAFB SLC 4E,-120.610829,34.632093,B5,1


In [25]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df_copy = spacex_df[['BoosterVersion','LaunchSite', 'Orbit', 'Latitude', 'Longitude', 'Class']].copy()
launch_sites_df = spacex_df_copy.groupby(['LaunchSite'], as_index=False).first()
launch_sites_df = launch_sites_df[['LaunchSite', 'Latitude', 'Longitude']]
launch_sites_df

,LaunchSite,Latitude,Longitude
0,CCSFS SLC 40,28.561857,-80.577366
1,KSC LC 39A,28.608058,-80.603956
2,VAFB SLC 4E,34.632093,-120.610829


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [26]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example,


In [27]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

In [28]:
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label
for site, lat, lng in zip(launch_sites_df['LaunchSite'], launch_sites_df['Latitude'], launch_sites_df['Longitude']):
    circle1 = folium.Circle([lat,lng], radius=50, color='Blue', fill=True).add_child(folium.Popup(site))
    marker1 = folium.map.Marker([lat,lng], icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % site))
    site_map.add_child(circle1)
    site_map.add_child(marker1)
    
site_map

Now, you can explore the map by zoom-in/out the marked areas
, and try to answer the following questions:

*   Are all launch sites in proximity to the Equator line?
*   Are all launch sites in very close proximity to the coast?

Also please try to explain your findings.


# Task 2: Mark the success/failed launches for each site on the map


Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [29]:
spacex_df_copy.tail(10)

,BoosterVersion,LaunchSite,Orbit,Latitude,Longitude,Class
122,Falcon 9,KSC LC 39A,LEO,28.608058,-80.603956,1
123,Falcon 9,VAFB SLC 4E,PO,34.632093,-120.610829,1
124,Falcon 9,CCSFS SLC 40,GTO,28.561857,-80.577366,1
125,Falcon 9,KSC LC 39A,ISS,28.608058,-80.603956,1
126,Falcon 9,KSC LC 39A,VLEO,28.608058,-80.603956,1
127,Falcon 9,CCSFS SLC 40,LEO,28.561857,-80.577366,1
128,Falcon 9,KSC LC 39A,VLEO,28.608058,-80.603956,1
129,Falcon 9,CCSFS SLC 40,SSO,28.561857,-80.577366,1
130,Falcon 9,VAFB SLC 4E,LEO,34.632093,-120.610829,1
131,Falcon 9,KSC LC 39A,VLEO,28.608058,-80.603956,1


Next, let's create markers for all launch records.
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Let's first create a `MarkerCluster` object


In [30]:
marker_cluster = MarkerCluster()

*TODO:* Create a new column in `launch_sites` dataframe called `marker_color` to store the marker colors based on the `class` value


In [32]:
# Function to assign color to launch outcome
def assign_marker_color(launch_outcome):
    if launch_outcome == 1:
        return 'green'
    else:
        return 'red'
    
spacex_df_copy['marker_color'] = spacex_df_copy['Class'].apply(assign_marker_color)
spacex_df_copy.tail(10)

,BoosterVersion,LaunchSite,Orbit,Latitude,Longitude,Class,marker_color
122,Falcon 9,KSC LC 39A,LEO,28.608058,-80.603956,1,green
123,Falcon 9,VAFB SLC 4E,PO,34.632093,-120.610829,1,green
124,Falcon 9,CCSFS SLC 40,GTO,28.561857,-80.577366,1,green
125,Falcon 9,KSC LC 39A,ISS,28.608058,-80.603956,1,green
126,Falcon 9,KSC LC 39A,VLEO,28.608058,-80.603956,1,green
127,Falcon 9,CCSFS SLC 40,LEO,28.561857,-80.577366,1,green
128,Falcon 9,KSC LC 39A,VLEO,28.608058,-80.603956,1,green
129,Falcon 9,CCSFS SLC 40,SSO,28.561857,-80.577366,1,green
130,Falcon 9,VAFB SLC 4E,LEO,34.632093,-120.610829,1,green
131,Falcon 9,KSC LC 39A,VLEO,28.608058,-80.603956,1,green


*TODO:* For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`


In [35]:
# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)
# for each row in spacex_df data frame
# create a Marker object with its coordinate
# and customize the Marker's icon property to indicate if this launch was successed or failed, 
# e.g., icon=folium.Icon(color='white', icon_color=row['marker_color']
for index, row in spacex_df_copy.iterrows():
    # TODO: Create and add a Marker cluster to the site map
    # marker = folium.Marker(...)
    marker = folium.Marker([row['Latitude'], row['Longitude']], popup=f'Rocket: {row["BoosterVersion"]},<br>  ---------- <br>Orbit:<br>{row["Orbit"]}', icon=folium.Icon(color='white', icon_color=row['marker_color']))
    marker_cluster.add_child(marker)
    
site_map

# TASK 3: Calculate the distances between a launch site to its proximities


Next, we need to explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [36]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Longitude: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


You can calculate the distance between two points on the map based on their `Lat` and `Long` values using the following method:


In [37]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

*TODO:* Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.


In [38]:
# find coordinate of the closet coastline
# e.g.,: Lat: 28.56367  Lon: -80.57163
# distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)
coast_lat = 28.56287
coast_lng = -80.56791
site_lat = launch_sites_df.iat[0,1]
site_lng = launch_sites_df.iat[0,2]

distance_coastline = calculate_distance(site_lat, site_lng, coast_lat, coast_lng)
distance_coastline

0.9306285669022478

*TODO:* After obtained its coordinate, create a `folium.Marker` to show the distance


In [39]:
# Create and add a folium.Marker on your selected closest coastline point on the map
# Display the distance between coastline point and launch site using the icon property 
# for example
distance_marker = folium.Marker([coast_lat, coast_lng], icon=DivIcon(icon_size=(20,20), icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f}KM".format(distance_coastline)))
site_map.add_child(distance_marker)

site_map

*TODO:* Draw a `PolyLine` between a launch site to the selected coastline point


In [40]:

# Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinate
lines=folium.PolyLine(locations=[[coast_lat, coast_lng],[site_lat, site_lng]], weight=1)
site_map.add_child(lines)

Your updated map with distance line should look like the following screenshot:


*TODO:* Similarly, you can draw a line betwee a launch site to its closest city, railway, highway, etc. You need to use `MousePosition` to find the their coordinates on the map first


A railway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png" />
</center>


A highway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png" />
</center>


A city map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png" />
</center>


In [41]:
# Create a marker with distance to a closest city, railway, highway, etc.
# Draw a line between the marker to the launch site
cocoa_lat = 28.35634
cocoa_lng = -80.73264
site_lat = launch_sites_df.iat[0,1]
site_lng = launch_sites_df.iat[0,2]

distance_city = calculate_distance(site_lat, site_lng, cocoa_lat, cocoa_lng)
distance_city

27.442972255628522

In [42]:
cocoa_marker = folium.Marker([cocoa_lat, cocoa_lng], icon=DivIcon(icon_size=(20,20), icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f}KM".format(distance_city)))
cocoa_line=folium.PolyLine(locations=[[cocoa_lat, cocoa_lng],[site_lat, site_lng]], weight=1)
site_map.add_child(cocoa_marker)
site_map.add_child(cocoa_line)
site_map

In [43]:
parkway_lat = 28.56246
parkway_lng = -80.57068
site_lat = launch_sites_df.iat[0,1]
site_lng = launch_sites_df.iat[0,2]

distance_parkway = calculate_distance(site_lat, site_lng, parkway_lat, parkway_lng)
distance_parkway

0.6566091653919113

In [44]:
parkway_marker = folium.Marker([parkway_lat, parkway_lng], icon=DivIcon(icon_size=(20,20), icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f}KM".format(distance_parkway)))
parkway_line=folium.PolyLine(locations=[[parkway_lat, parkway_lng],[site_lat, site_lng]], weight=1)
site_map.add_child(parkway_marker)
site_map.add_child(parkway_line)

In [45]:
railroad_lat = 28.57322
railroad_lng = -80.58522
site_lat = launch_sites_df.iat[0,1]
site_lng = launch_sites_df.iat[0,2]

distance_railroad = calculate_distance(site_lat, site_lng, railroad_lat, railroad_lng)
distance_railroad

1.4785421582250513

In [46]:
railroad_marker = folium.Marker([railroad_lat, railroad_lng], icon=DivIcon(icon_size=(20,20), icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f}KM".format(distance_railroad)))
railroad_line=folium.PolyLine(locations=[[railroad_lat, railroad_lng],[site_lat, site_lng]], weight=1)
site_map.add_child(railroad_marker)
site_map.add_child(railroad_line)

After you plot distance lines to the proximities, you can answer the following questions easily:

*   Are launch sites in close proximity to railways?
*   Are launch sites in close proximity to highways?
*   Are launch sites in close proximity to coastline?
*   Do launch sites keep certain distance away from cities?

Also please try to explain your findings.


In [48]:
project.save_data(data=spacex_df_copy.to_csv(index=False), file_name='sites_folium.csv', overwrite=True)

{'file_name': 'sites_folium.csv',
 'message': 'File saved to project storage.',
 'bucket_name': 'applieddatasciencecapstone-donotdelete-pr-jx5un88ldtszdu',
 'asset_id': 'da0429a9-ae71-458b-8aab-d90d8776e620'}

# Next Steps:

Now you have discovered many interesting insights related to the launch sites' location using folium, in a very interactive way. Next, you will need to build a dashboard using Ploty Dash on detailed launch records.


## Authors


[Yan Luo](https://www.linkedin.com/in/yan-luo-96288783/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2021-01-01)


### Other Contributors


Joseph Santarcangelo


## Change Log


| Date (YYYY-MM-DD) | Version | Changed By | Change Description          |
| ----------------- | ------- | ---------- | --------------------------- |
| 2021-05-26        | 1.0     | Yan        | Created the initial version |


Copyright © 2021 IBM Corporation. All rights reserved.
